## **Prepare the use of KQL magic in Jupyter Notebooks**

In order to leverage the use of KQL magic in Notebooks, you need to install the KQL magic extenstion.

It does not matter where you install it, it wil work for both Azure Data Explorer and other notebook tools you might be using.

In [ ]:
import sys
!{sys.executable} -m pip install Kqlmagic --no-cache-dir --upgrade

In [ ]:
import sys
!{sys.executable} -m pip list

## **Enable the KQL extension**

After installation, you need to enable the extension in the current application.

In [ ]:
%reload_ext Kqlmagic

## **Connect to KQL Cluster**

Now you can connect to your KQL cluster - either by using default login (with popup) or by using application key approach

In [2]:
## Default pop-up approach
%kql AzureDataExplorer://code;cluster='https://trd-kkqsa5kpw8x15701xn.z3.kusto.fabric.microsoft.com';database='7a4b04a5-ed06-4860-87da-a5af20b2392c'

UsageError: Line magic function `%kql` not found.


In [ ]:
## application key approach
%kql AzureDataExplorer://tenant='<tenant-id>';clientid='<aad-appid>';clientsecret='<aad-appkey>';cluster='<cluster-name>';database='<database-name>'

In [ ]:
## Connect to the HELP cluster from Microsoft
%kql AzureDataExplorer://tenant="Microsoft.com";code;cluster='help';database='Samples'

## **Begin to use your Kusto magic**

After connection you can begin to leverage your kusto abilities and get some things done

## **In the SQL world:**

select \* from NYCTaxi

In [ ]:
%%kql

NYCTaxi
| take 100

## **In the SQL world:**

<span style="color: var(--vscode-foreground);">select * from NYCTaxi<br></span><span style="color: var(--vscode-foreground);">where VentorID = 2</span>

In [ ]:
%%kql

NYCTaxi
| where VendorID == 2
| take 100


## **In the SQL world:**

<span style="color: var(--vscode-foreground);">select * from NYCTaxi<br></span><span style="color: var(--vscode-foreground);">where VentorID = 2<br></span><span style="color: var(--vscode-foreground);">order by passenger_count</span>

In [ ]:
%%kql

NYCTaxi
| where VendorID == 2
| take 100
| order by passenger_count


## **In the SQL world:**

<span style="color: var(--vscode-foreground);">select count(*) from NYCTaxi</span>

In [ ]:
%%kql

NYCTaxi
| count

## **In the SQL world:**

select   
    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; passenger_count<br></span>    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; ,VendorID<br></span>    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; ,trip_distance<br></span><span style="color: var(--vscode-foreground);">from NYCTaxi</span>

In [ ]:
%%kql

NYCTaxi
| project passenger_count, VendorID, trip_distance


In the SQL world:

  

select   
    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; passenger_count<br></span>    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; ,VendorID<br></span>    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; ,trip_distance<br></span>    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; ,total_amount / passenger_count as AmtPsngr<br></span><span style="color: var(--vscode-foreground);">from NYCTaxi</span>

In [ ]:
%%kql

NYCTaxi
| take 100
| extend AmtPsngr = total_amount / passenger_count
| project passenger_count, VendorID, trip_distance, AmtPsngr


## **In the SQL world:**

select   
    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; sum(passenger_count) as SumPassenger<br></span>    <span style="color: var(--vscode-foreground);">&nbsp; &nbsp; ,VendorID<br></span><span style="color: var(--vscode-foreground);">from NYCTaxi<br></span><span style="color: var(--vscode-foreground);">group by VendorID</span>

In [ ]:
%%kql

NYCTaxi
| summarize SumPassenger = sum(passenger_count) by VendorID


## **Putting it all together**

Please note that I can use several WHERE clauses and other functions whereever I need.

In [ ]:
%%kql

NYCTaxi
| where passenger_count > 1
| project passenger_count, total_amount, VendorID, fare_amount
| extend AmtPsngr = total_amount / passenger_count
| where AmtPsngr > 10
| summarize TotalAmount = sum(total_amount), AvgAmtPsngr = avg(AmtPsngr) by VendorID
| where VendorID  <> 1


## **Rendering**

<span style="color: var(--vscode-foreground);">I can also render the results directly in the Notebook or in the query editor in Fabric</span>

In [ ]:
%%kql

NYCTaxi
| where tpep_pickup_datetime between (datetime(2009-01-01)..datetime(2015-01-01))
| extend PickUpdate = startofday(tpep_pickup_datetime)
| summarize SumPsngrCount = sum(passenger_count) by PickUpdate
| project PickUpdate, SumPsngrCount
| render  timechart
    with(
        title = "timechart"
        ,xtitle = "Time"
        ,ytitle = "Fares"
    )


## **Use resultset in next cell**

After you have your data as you wish, you can begin to work with it in dataframes as you would any other dataset

In [ ]:
%%kql

NYCTaxi
| summarize count=count() by HourOfDay=hourofday(tpep_pickup_datetime)
| order by HourOfDay

In [ ]:
df = _kql_raw_result_.to_dataframe()
df